In [ ]:
!pip install pytorch-tabnet

In [ ]:
import os
import glob
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
import pandas as pd
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
class RowwiseDataset(Dataset):
    def __init__(self, X_folder, y_attack_folder):
        self.X_data = pd.concat(
            [pd.read_parquet(f) for f in sorted(glob.glob(os.path.join(X_folder, "*.parquet")))]
        ).values
        self.y_attack_data = pd.concat(
            [pd.read_parquet(f)["Attack"] for f in sorted(glob.glob(os.path.join(y_attack_folder, "*.parquet")))]
        ).values
        assert len(self.X_data) == len(self.y_attack_data), \
            "Mismatched row counts between features and labels."
        assert self.y_attack_data.ndim == 1, "Target data should be 1D after column selection."

    def __len__(self):
        return len(self.X_data)

    def __getitem__(self, idx):
        return (
            torch.tensor(self.X_data[idx], dtype=torch.float32),
            torch.tensor(self.y_attack_data[idx], dtype=torch.float32),
        )

In [ ]:
X_train_folder = "/kaggle/input/nf-ton-iot-v2-cleaned-split/rmv_outlier_std_801010/X_train"
y_train_attack_folder = "/kaggle/input/nf-ton-iot-v2-cleaned-split/rmv_outlier_std_801010/y_train_attack"

X_valid_folder = "/kaggle/input/nf-ton-iot-v2-cleaned-split/rmv_outlier_std_801010/X_valid"
y_valid_attack_folder = "/kaggle/input/nf-ton-iot-v2-cleaned-split/rmv_outlier_std_801010/y_valid_attack"

X_test_folder = "/kaggle/input/nf-ton-iot-v2-cleaned-split/rmv_outlier_std_801010/X_test"
y_test_attack_folder = "/kaggle/input/nf-ton-iot-v2-cleaned-split/rmv_outlier_std_801010/y_test_attack"

In [ ]:
train_dataset = RowwiseDataset(X_train_folder, y_train_attack_folder)
valid_dataset = RowwiseDataset(X_valid_folder, y_valid_attack_folder)
test_dataset = RowwiseDataset(X_test_folder, y_test_attack_folder)

X_train, y_train_attack = train_dataset.X_data, train_dataset.y_attack_data
X_valid, y_valid_attack = valid_dataset.X_data, valid_dataset.y_attack_data
X_test, y_test_attack = test_dataset.X_data, test_dataset.y_attack_data

y_train = y_train_attack
y_valid = y_valid_attack
y_test = y_test_attack

In [ ]:
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_valid shape: {X_valid.shape}, y_valid shape: {y_valid.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

In [ ]:
class WeightedCrossEntropyLoss(nn.Module):
    def __init__(self, cls_num_list, reduction='mean'):
        super(WeightedCrossEntropyLoss, self).__init__()
        total_samples = sum(cls_num_list)
        self.class_weights = torch.tensor([total_samples / c for c in cls_num_list])
        self.reduction = reduction

    def forward(self, inputs, targets):
        log_probs = F.log_softmax(inputs, dim=1)
        weights = self.class_weights.to(inputs.device)
        loss = F.nll_loss(log_probs, targets, weight=weights, reduction=self.reduction)
        return loss

In [ ]:
cls_num_list = [4357259, 2943073, 1001429, 961486, 480101, 152251, 125024, 4149, 427, 109]
weighted_ce_loss = WeightedCrossEntropyLoss(cls_num_list=cls_num_list)

In [ ]:
sizes = [64]
results = []

for size in sizes:
    print(f"Training TabNet with n_a=n_d={size}")

    clf = TabNetClassifier(
        device_name='auto',
        n_d=size,
        n_a=size,
        lambda_sparse=0,
        mask_type='entmax',
        optimizer_params=dict(lr=1e-2, weight_decay=1e-5),
        scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
        scheduler_params={
            "mode": "min",
            "factor": 0.5,
            "patience": 10,
            "min_lr": 1e-5,
        },
        verbose=1,
        seed=42
    )

    clf.fit(
        X_train,
        y_train,
        eval_set=[(X_valid, y_valid)],
        eval_metric=["balanced_accuracy", "accuracy"],
        max_epochs=100000,
        patience=30,
        batch_size=1024 * 10,
        virtual_batch_size=128 * 10,
        loss_fn=weighted_ce_loss,
        compute_importance=False,
    )

    model_path = f"weighted_cross_entropy_acc_focused_tabnet_model_size_{size}.zip"
    clf.save_model(model_path)

    y_pred = clf.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')

    predictions_path = f"weighted_cross_acc_focused_entropy_y_pred_size_{size}.csv"
    pd.DataFrame(y_pred, columns=["y_pred"]).to_csv(predictions_path, index=False)

    results.append({
        "size": size,
        "accuracy": acc,
        "f1_score": f1,
        "model_path": model_path,
        "predictions_path": predictions_path,
    })

results_df = pd.DataFrame(results)
results_df.to_csv("weighted_cross_entropy_acc_focused_results_summary.csv", index=False)

print("Tuning completed. Models, predictions, and results have been saved.")